# ライブラリ準備

In [31]:
#coding:utf-8
import pylab
from PIL import Image,ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
import copy
from sklearn.decomposition import PCA
# from multiprocessing import Pool
# import multiprocessing as multi
from joblib import Parallel, delayed
from time import time
import scipy.linalg

img_size=20
#画像の読み込み
test = cv2.imread("./Img/input.jpg", cv2.IMREAD_COLOR)#BGRなので気をつける
gray_test = cv2.imread("./Img/input.jpg",cv2.IMREAD_GRAYSCALE)
width = test.shape[1]
height = test.shape[0] 

#画像の書き出し
cv2.imwrite('./Img/test.jpg', test)
cv2.imwrite('./Img/gray_test.jpg',gray_test)

True

# テスト形状取得

In [32]:
#####学習用テスト形状#####
shape = np.zeros((24,2))
bef_shape = np.zeros((24,2))
shape[:,0] = [12,97,24,62,56,42,96,30,125,34,151,48,178,94,174,137,136,158,100,165,62,158,32,136]
shape[:,1] = [49,173,27,143,24,105,46,57,78,31,114,23,150,36,175,73,172,113,150,148,120,172,85,185]
#####ここらへんは20x20のサイズに変更したもので、テストケースとして小さくしている###
shape = shape/10
shape = np.round(shape)
print(shape)
unchanged_shape = copy.deepcopy(shape)
gray_shape_arr =list()

black = np.zeros((img_size,img_size))
N = int(len(shape[:,0])/2)
for i in range(0,int(N)):
    x = int(shape[2*i][0])
    y = int(shape[2*i+1][0])
    x2 = int(shape[(2*(i+1))%(2*N)][0])
    y2 = int(shape[(2*(i+1)+1)%(2*N)][0])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 1)
cv2.imwrite('./Img/black_shape0.jpg' ,black)

gray_shape0_img = cv2.imread("./Img/Samples/shape0.jpg",cv2.IMREAD_GRAYSCALE)
gray_shape_arr.append(gray_shape0_img)
cv2.imwrite('./Img/gray_shape0.jpg' ,gray_shape_arr[0])

black = np.zeros((img_size,img_size))
N = int(len(shape[:,1])/2)
for i in range(0,int(N)):
    x = int(shape[2*i][1])
    y = int(shape[2*i+1][1])
    x2 = int(shape[(2*i+2)%(2*N)][1])
    y2 = int(shape[(2*i+3)%(2*N)][1])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 1)
cv2.imwrite('./Img/black_shape1.jpg' ,black)

gray_shape1_img = cv2.imread("./Img/Samples/shape1.jpg",cv2.IMREAD_GRAYSCALE)
gray_shape_arr.append(gray_shape1_img)
cv2.imwrite('./Img/gray_shape1.jpg' ,gray_shape_arr[1])

#####平均形状取得######
black = np.zeros((img_size,img_size))
shape_ave = (shape[:,0]+shape[:,1])/2 #平均形状
for i in range(0,int(N)):
    x = int(shape_ave[2*i])
    y = int(shape_ave[2*i+1])
    x2 = int(shape_ave[(2*(i+1))%(2*N)])
    y2 = int(shape_ave[(2*(i+1)+1)%(2*N)])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 1)
cv2.imwrite('./Img/black_shape_ave.jpg' ,black)



[[  1.   5.]
 [ 10.  17.]
 [  2.   3.]
 [  6.  14.]
 [  6.   2.]
 [  4.  10.]
 [ 10.   5.]
 [  3.   6.]
 [ 12.   8.]
 [  3.   3.]
 [ 15.  11.]
 [  5.   2.]
 [ 18.  15.]
 [  9.   4.]
 [ 17.  18.]
 [ 14.   7.]
 [ 14.  17.]
 [ 16.  11.]
 [ 10.  15.]
 [ 16.  15.]
 [  6.  12.]
 [ 16.  17.]
 [  3.   8.]
 [ 14.  18.]]


True

# 外観の正規化

In [33]:
gray_rot_arr = list()#正規化された外観
for j in range(0,len(gray_shape_arr)):
    N = int(len(shape[:,j])/2)
    #平均形状の重心
    g_ave = np.array([0,0])
    g_ave = np.sum(shape_ave[::2])/N,np.sum(g_ave[1::2])/N
    #形状の重心
    g_shape = np.array([0,0])
    g_shape = np.sum(shape[::2,j])/N,np.sum(shape[1::2,j])/N
    
    shape[::2,j] -= g_shape[0]
    shape[1::2,j] -= g_shape[1]
    #パラメータ計算
    param_a = np.dot(shape[:,j],shape_ave[:])/np.dot(shape[:,j],shape[:,j])
    param_b = 0
    loop = int(shape[:,j].shape[0]/2)
    for i in range(0,loop):
        x1_ = shape[2*i,j]
        y1_ = shape[2*i+1,j]
        x2_ = shape_ave[2*i]
        y2_ = shape_ave[2*i+1]
        param_b += (x1_*y2_-x2_*y1_ )
    param_b /= np.dot(shape[:,j],shape[:,j])
    param_s = (param_a**2+param_b**2)**(1/2)
    param_theta = np.arctan2(param_b,param_a)
    print("拡大率s:",param_s,",回転角θ:",param_theta)
    # 回転変換行列の算出
    rotation_matrix = cv2.getRotationMatrix2D((img_size/2,img_size/2), param_theta*180/math.pi ,param_s)#現在(20,20)のみにつくってる
    size = tuple((img_size,img_size))
    img_rot = cv2.warpAffine(gray_shape_arr[j], rotation_matrix, size, flags=cv2.INTER_CUBIC)
    gray_rot_arr.append(img_rot)
    cv2.imwrite("./Img/gray_shape"+str(j)+"_rot.jpg" ,img_rot)


拡大率s: 0.916888828299 ,回転角θ: -0.418701878127
拡大率s: 0.904101229794 ,回転角θ: 0.412443631354


# 平均外観の取得

In [34]:
appearance_ave = np.zeros((img_size,img_size))
for val in gray_rot_arr:
    appearance_ave += val
appearance_ave /=  len(gray_rot_arr)
cv2.imwrite("./Img/appearance_ave.jpg" ,appearance_ave)


True

# 主成分分析
http://www.cvlab.cs.tsukuba.ac.jp/index.php?plugin=attach&pcmd=open&file=imgrec2016-handout-5.pdf&refer=2016%C7%AF%C5%D9%20%B2%E8%C1%FC%C7%A7%BC%B1%B9%A9%B3%D8

In [48]:
shape_sample_num  = shape.shape[1]

# S = np.zeros((2*N,2*N))
# for i in range(0,2*N):
#     for j in range(0,2*N):
#         for n in range(0,shape_sample_num):
#             S[i,j] += (shape[i][n]-shape_ave[i])*(shape[j][n]-shape_ave[j])
# S /= shape_sample_num-1

pca = PCA()
shape = np.transpose(unchanged_shape)
pca.fit(shape)
S_mat = pca.get_covariance()
print("S_mat covariance")
la,v = scipy.linalg.eigh(S_mat)
print("laとv")
la = la[::-1]
v = v[::-1]

la_sum = np.sum(la)
need_la = 0 #必要固有値の和
need_la_num = 1 #必要固有値数-1
for val in la:
    need_la += val
    if(la_sum*0.98 < need_la):
        break
    else:
        need_la_num +=1
   
S_P = np.empty((shape_ave.shape[0],need_la_num))
for j in range(0,need_la_num):
    S_P[:,j] = v[:,j]
S_b = np.empty((need_la_num))
print("形状固有ベクトル行列",S_P)
print("形状固有値",S_b)


#輝度値共分散行列
appearance_list = list()
for i in range(len(gray_rot_arr)):
    appearance_list.append(np.ravel(gray_rot_arr[i]))#外観行列をベクトル化
appearance = np.array(appearance_list)
pca.fit(appearance)
A_mat = pca.get_covariance()
print("A_mat covariance")
la,v = scipy.linalg.eigh(A_mat)
print("laとv")
la = la[::-1]
v = v[::-1]
la_sum = np.sum(la)
need_la = 0 #必要固有値の和
need_la_num = 1 #必要固有値数-1
for val in la:
    need_la += val
    if(la_sum*0.98 < need_la):
        break
    else:
        need_la_num +=1

A_P = np.empty((N**2,need_la_num))
for j in range(0,need_la_num):
    A_P[:,j] = v[:,j]
A_b = np.empty((need_la_num))
print("外観固有ベクトル行列",A_P)
print("外観固有値",A_b)


# appearance_ave_vec = np.ravel(appearance_ave)

#######ここまでPCA#####
# def aCalc(i,j): #外観の共分散行列を求める
#     A_cell = 0 
#     for n in range(0,shape_sample_num):
#         A_cell += (appearance_list[n][i]-appearance_ave_vec[i])*(appearance_list[n][j]-appearance_ave_vec[j])
#     A_cell /= shape_sample_num-1
#     print(i,j)
#     return A_cell

# def aCalcWrapper(args):
#     return aCalc(*args)

# def process(i,j):
#     return i*j

# pic_size = 30
# result = Parallel(n_jobs=-1,verbose=0)([delayed(aCalc)(i,j) for i in range(0,pic_size**2) for j in range(0,pic_size**2)])
# result = np.array(result)
# result = result.reshape(pic_size**2,pic_size**2)
# print(result)
# p = Pool(multi.cpu_count())
# index_list = [ [i,j] for  i in range(0,200*200) for j in range(0,200*200)] 
# A_cell_list = p.map(aCalc,index_list)
# p.close()
# A_cell_array = np.array(A_cell_list)
# A_mat = A_cell_array.reshape(200,200)
# print(A_mat)

# A_mat = np.zeros((200*200,200*200))
# for i in range(0,200**2):
#     print(i)
#     for j in range(0,200**2):
#         for n in range(0,shape_sample_num):
#             A_mat[i,j] += (appearance_list[n][i]-appearance_ave_vec[i])*(appearance_list[n][j]-appearance_ave_vec[j])
# A_mat /= shape_sample_num-1


# ma = np.max(S)
# S /= ma
# S *= 255
# cv2.imwrite('./Img/s.jpg' ,S)
# ma = np.max(A_mat)
# A_mat /= ma
# A_mat *= 255
# cv2.imwrite('./Img/a_mat.jpg' ,A_mat)

S_mat covariance
laとv
形状固有ベクトル行列 [[-0.18384017]
 [ 0.30278695]
 [-0.04596004]
 [ 0.31600326]
 [ 0.10513639]
 [-0.17062386]
 [ 0.17062386]
 [ 0.15800163]
 [-0.09251416]
 [-0.04596004]
 [-0.30278695]
 [ 0.07870378]
 [ 0.07870378]
 [ 0.18384017]
 [ 0.        ]
 [ 0.18384017]
 [ 0.10298611]
 [ 0.25938839]
 [ 0.20597223]
 [ 0.21481529]
 [-0.42963058]
 [-0.05370382]
 [ 0.37689522]
 [ 0.        ]]
形状固有値 [ 239.5]
A_mat covariance
laとv
外観固有ベクトル行列 [[ -3.77286119e-02]
 [ -3.61797573e-02]
 [ -2.37911752e-02]
 [  2.67709071e-03]
 [  2.53793076e-02]
 [  5.45512891e-02]
 [  2.25676057e-02]
 [ -2.40945932e-02]
 [  5.13342871e-02]
 [ -8.53128202e-02]
 [  9.36565482e-02]
 [ -6.89221415e-02]
 [  2.64100126e-02]
 [  1.24533592e-02]
 [ -4.74618079e-04]
 [  7.74677253e-02]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  0.00000000e+00]
 [  5.09922598e-02]
 [  5.07243692e-02]
 [  3.83076924e-02]
 [ -9.30964009e-02]
 [  8.79859193e-03]
 [  5.14842084e-02]
 [ -6.04422798e-02]
 [  1.59957511e-02

In [ ]:
形状固有ベクトル行列 [[-0.16668358]
 [-0.34237708]
 [-0.01351488]
 [-0.36490189]
 [ 0.14415877]
 [-0.28381258]
 [ 0.22524808]
 [-0.12163396]
 [ 0.21173319]
 [ 0.01351488]
 [ 0.16668358]
 [ 0.11262404]
 [ 0.12613892]
 [ 0.26128777]
 [-0.00450496]
 [ 0.28831754]
 [-0.16217862]
 [ 0.20272327]
 [-0.22524808]
 [ 0.07658435]
 [-0.26128777]
 [-0.06306946]
 [-0.23876296]
 [-0.22074312]]

In [ ]:
形状固有値 [ 24637.]

In [30]:
param=2#(±3までらしいが、、、)
for i in range(0,len(b)):
    b[i] = la[i]**(1/2)*param
# print(P.shape,b.shape,result.shape)
# print(P)
# print(b)
result = shape_ave+np.dot(P,b)

# print(result)
# 輪郭線のテスト
black_result = np.zeros((img_size,img_size))
N = int(len(shape[:,0])/2)
for i in range(0,int(N)):
    x = int(result[2*i])
    y = int(result[2*i+1])
    x2 = int(result[(2*(i+1))%(2*N)])
    y2 = int(result[(2*(i+1)+1)%(2*N)])
    print(x,y,x2,y2)
    black_result = cv2.line(black_result, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/black_result.jpg' ,black_result)

NameError: name 'b' is not defined

In [26]:
#test
hoge= np.zeros((4,2))
fuga_  = np.zeros((4,2))
fuga_[:,0] =  [1,1,1,2]
fuga_[:,1] = [3,3,4,4]
hoge[:,0] = copy.deepcopy(fuga_[:,0])
hoge[:,1] = copy.deepcopy(fuga_[:,1])
print("移動前:\n",hoge)
#重心とる
N = int(len(hoge[:,0])/2)
g1_ = np.array([0,0])
g1_ = np.sum(hoge[::2,0])/N,np.sum(hoge[1::2,0])/N
g2_ = np.array([0,0])
g2_ = np.sum(hoge[::2,1])/N,np.sum(hoge[1::2,1])/N
hoge[::2,0] -= g1_[0]
hoge[1::2,0] -= g1_[1]
hoge[::2,1] -= g2_[0]
hoge[1::2,1] -= g2_[1]
print(g2_)
print("移動後:\n",hoge)
#パラメータ計算
a_ = np.dot(hoge[:,0],hoge[:,1])/np.dot(hoge[:,0],hoge[:,0])
b_ =0
loop = int(hoge[:,0].shape[0]/2)
for i in range(0,loop):
    x1_ = hoge[2*i,0]
    y1_ = hoge[2*i+1,0]
    x2_ = hoge[2*i,1]
    y2_ = hoge[2*i+1,1]
    b_ += (x1_*y2_-x2_*y1_ )
b_ /= np.dot(hoge[:,0],hoge[:,0])
s_ = (a_**2+b_**2)**(1/2)
theta_ = np.arctan2(b_,a_)
print("s:",s_,"θ:",theta_)


移動前:
 [[ 1.  3.]
 [ 1.  3.]
 [ 1.  4.]
 [ 2.  4.]]
(3.5, 3.5)
移動後:
 [[ 0.  -0.5]
 [-0.5 -0.5]
 [ 0.   0.5]
 [ 0.5  0.5]]
s: 1.41421356237 θ: -0.785398163397


NameError: name 'center' is not defined

In [385]:
sA_ =  np.zeros((2,2))
sA_[:,0] = [s_*np.cos(theta_),-s_*np.sin(theta_)]
sA_[:,1] = [s_*np.sin(theta_), s_*np.cos(theta_)]

ans_ = [0,0,0,0]
for i in range(0,loop):
    aho = [hoge[2*i,0],hoge[2*i+1,0]]
    ans_[2*i],ans_[2*i+1] = np.dot(aho,sA_) +g2_




In [386]:
ans_,fuga_[:,1]

([3.0, 3.0, 4.0, 4.0], array([ 3.,  3.,  4.,  4.]))

In [21]:
list(range(2))

[0, 1]

In [19]:
list([1,2])

[1, 2]

In [27]:
aho = [ [i,j] for  i in range(0,200) for j in range(0,200)] 

In [34]:
print(type(aho))
print(np.array(aho))

<class 'list'>
[[  0   0]
 [  0   1]
 [  0   2]
 ..., 
 [199 197]
 [199 198]
 [199 199]]


In [21]:
for x in dir(pca):
  print(x)

__abstractmethods__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__setstate__
__sizeof__
__str__
__subclasshook__
__weakref__
_abc_cache
_abc_negative_cache
_abc_negative_cache_version
_abc_registry
_fit
_fit_full
_fit_truncated
_get_param_names
components_
copy
explained_variance_
explained_variance_ratio_
fit
fit_transform
get_covariance
get_params
get_precision
inverse_transform
iterated_power
mean_
n_components
n_components_
n_features_
n_samples_
noise_variance_
random_state
score
score_samples
set_params
singular_values_
svd_solver
tol
transform
whiten
